In [1]:
import nltk
from os import getcwd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import numpy as np
from nltk.corpus import twitter_samples

In [2]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/sadra/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sadra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# add folder, tmp2, from our local workspace containing pre-downloaded corpora files to nltk's data path
# this enables importing of these files without downloading it again when we refresh our workspace

filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [4]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [5]:
## Processing Tweets
def process_tweets(tweet):
    ## initiat stemming class:
    stemmer = PorterStemmer()
    ## remove old style retwee text "RT"
    tweet = re.sub(r'^RT[\s]', '', tweet)
    ## remove  hyper link
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    ### only rmove the # sighn from the word
    tweet = re.sub(r'#','', tweet)
    ## instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    ## Tokenize tweets
    tweet_tokens = tokenizer.tokenize(tweet)
    ## import English stop words list form NLTK
    stopwords_english = stopwords.words('english')
    
    # creat a list of words without stopwords
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
               word not in string.punctuation): ## remove punctuation
            stem_word = stemmer.stem(word)  ## stemming word
            tweets_clean.append(stem_word)
#     print('tweets_clean: ', tweets_clean)
    return tweets_clean
        
    

In [6]:
#Frequency generation function
def build_freqs(tweets, ys):

    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
#     print('ys: ', ys) 
    
    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
  
    ylist = np.squeeze(ys).tolist()
#     print('ylist: ', ylist)
    freqs = {}
    for y, tweet in zip(ylist, tweets):
        for word in process_tweets(tweet):
            pair = (word,y)
            if pair in freqs:
                freqs[pair] =+ 1
            else:
                freqs[pair] = 1
    return freqs    


In [7]:
def sigmoid(z):
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    # calculate the sigmoid of z
    h = 1/(1+np.exp(-z))
    return h

In [8]:
def gradientDescent(x,y, theta, alpha, num_iters):
    
    '''Input:
        x: matrix of feature which is (m, n+1)
        y: corresponding label of the input matrix x, dimmensions (m,1)
        theta: weigh vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of  iterations you to train your model for
        
        Output:
        J: the final cost
        theta: your final  weigh vector
        Hint: you might want to print cost to make sure that it is going down
    '''
    m = x.shape[0]
    
    for i in range(0,num_iters):
        # get z, the dot producto of x and theta
        z = np.dot(x, theta)
        # get the sigmoid of z
        h = sigmoid(z)
        
        # calculate the function
        J = (-1/m)*(np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)))
        
        # update the weighs theta
        theta = theta -(alpha/m)*np.dot(x.T, h-y)
        J = float(J)
        return J, theta

In [9]:
def extract_features(tweet, freqs):
    '''
    Input:
    tweet a list of words for one tweet
    freqs: dictionary corresponding to the frequencies of each tuple(word,label)
    
    Output:
    x: a feature vector of dimmension (1,3)
    '''
    # process_tweet, tokenizes, stems, and remove stopwords
    word_l = process_tweets(tweet)
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1,3))
    # bisa term is set to 1
    x[0,0] = 1
    
    # loop through each word in the list of words
    for word in word_l:
        # increment the word count for the positive label 1
        if freqs.get((word, 1.0))!=None:
            x[0,1] += freqs.get((word, 1.0))
        
        #increment the word count for label 0
        if freqs.get((word, 0.0))!=None:
            x[0,2] += freqs.get((word, 0.0))
    
    assert(x.shape==(1,3))
    return x

In [10]:
## split data into two pieces, one for training and for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]
train_x = train_pos + train_neg
test_x = test_pos + test_neg

# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos),1)), np.zeros((len(train_neg),1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg),1)), axis=0)
# combine positive and negative labels

In [18]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [11]:
freqs = build_freqs(train_x, train_y)
freqs

{('followfriday', 1.0): 1,
 ('top', 1.0): 1,
 ('engag', 1.0): 1,
 ('member', 1.0): 1,
 ('commun', 1.0): 1,
 ('week', 1.0): 1,
 (':)', 1.0): 1,
 ('hey', 1.0): 1,
 ('jame', 1.0): 1,
 ('odd', 1.0): 1,
 (':/', 1.0): 1,
 ('pleas', 1.0): 1,
 ('call', 1.0): 1,
 ('contact', 1.0): 1,
 ('centr', 1.0): 1,
 ('02392441234', 1.0): 1,
 ('abl', 1.0): 1,
 ('assist', 1.0): 1,
 ('mani', 1.0): 1,
 ('thank', 1.0): 1,
 ('listen', 1.0): 1,
 ('last', 1.0): 1,
 ('night', 1.0): 1,
 ('bleed', 1.0): 1,
 ('amaz', 1.0): 1,
 ('track', 1.0): 1,
 ('scotland', 1.0): 1,
 ('congrat', 1.0): 1,
 ('yeaaah', 1.0): 1,
 ('yipppi', 1.0): 1,
 ('accnt', 1.0): 1,
 ('verifi', 1.0): 1,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 1,
 ('blue', 1.0): 1,
 ('tick', 1.0): 1,
 ('mark', 1.0): 1,
 ('fb', 1.0): 1,
 ('profil', 1.0): 1,
 ('15', 1.0): 1,
 ('day', 1.0): 1,
 ('one', 1.0): 1,
 ('irresist', 1.0): 1,
 ('flipkartfashionfriday', 1.0): 1,
 ('like', 1.0): 1,
 ('keep', 1.0): 1,
 ('love', 1.0): 1,
 ('custom', 1.0): 1,
 ('wait',

In [12]:
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)


[[1. 7. 5.]]


In [13]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.69314718.
The resulting vector of weights is [0.0, 0.0, -0.0]


In [16]:
# prediction
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    # extract of the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    print('x: ', x)
    y_pred = sigmoid(np.dot(x,theta))
    
    print('y_pred: ', y_pred)
    return y_pred

In [17]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %.3f' % (tweet, predict_tweet(tweet, freqs, theta)))    

x:  [[1. 1. 1.]]
y_pred:  [[0.5]]
I am happy -> 0.500
x:  [[1. 1. 1.]]
y_pred:  [[0.5]]
I am bad -> 0.500
x:  [[1. 2. 2.]]
y_pred:  [[0.5]]
this movie should have been great. -> 0.500
x:  [[1. 1. 1.]]
y_pred:  [[0.5]]
great -> 0.500
x:  [[1. 2. 2.]]
y_pred:  [[0.5]]
great great -> 0.500
x:  [[1. 3. 3.]]
y_pred:  [[0.5]]
great great great -> 0.500
x:  [[1. 4. 4.]]
y_pred:  [[0.5]]
great great great great -> 0.500
